In [30]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType,MapType}
import org.apache.spark.sql.{DataFrame}
import org.apache.spark.sql.functions
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.{Pipeline,PipelineModel}
import org.apache.spark.ml.feature.Imputer

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType, MapType}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.Imputer


In [2]:
val spark = SparkSession
    .builder()
    .appName("Utils")
    .master("local[2]")
    .getOrCreate()

val sc = spark.sparkContext

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4e2b0c1a
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@f4461f8


# Dataframe Array数据类型处理

In [21]:
var simpleArrayDF = Seq(("beatles", "help,hey jude,some time"),
                        ("romeo", "eres mia,hahah,check")
                        ).toDF("name","songs")

simpleArrayDF = simpleArrayDF.withColumn(
    "hit_songs",functions.split(col("songs"),"\\,")
)

simpleArrayDF: org.apache.spark.sql.DataFrame = [name: string, songs: string ... 1 more field]
simpleArrayDF: org.apache.spark.sql.DataFrame = [name: string, songs: string ... 1 more field]


In [22]:
simpleArrayDF.printSchema()

root
 |-- name: string (nullable = true)
 |-- songs: string (nullable = true)
 |-- hit_songs: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [23]:
simpleArrayDF.show()

+-------+--------------------+--------------------+
|   name|               songs|           hit_songs|
+-------+--------------------+--------------------+
|beatles|help,hey jude,som...|[help, hey jude, ...|
|  romeo|eres mia,hahah,check|[eres mia, hahah,...|
+-------+--------------------+--------------------+



## 把Array里的内容展开

In [27]:
simpleArrayDF.select($"name",functions.explode($"hit_songs").as("hit_songs")).show()

+-------+---------+
|   name|hit_songs|
+-------+---------+
|beatles|     help|
|beatles| hey jude|
|beatles|some time|
|  romeo| eres mia|
|  romeo|    hahah|
|  romeo|    check|
+-------+---------+



## Array 是否包含某个元素

In [49]:
simpleArrayDF.withColumn(
    "is_contained",
    functions.array_contains($"hit_songs","help")
).show()

+-------+--------------------+--------------------+------------+
|   name|               songs|           hit_songs|is_contained|
+-------+--------------------+--------------------+------------+
|beatles|help,hey jude,som...|[help, hey jude, ...|        true|
|  romeo|eres mia,hahah,check|[eres mia, hahah,...|       false|
+-------+--------------------+--------------------+------------+



# Dataframe Map 数据类型处理

In [43]:
var simpleMapDataFrame = Seq(
        ("sublime", Map("good_song" -> "santeria","bad_song" -> "doesn't exist")),
        ("prince_royce", Map("good_song" -> "darte un beso","bad_song" -> "back it up"))
        ).toDF("name","songs")

simpleMapDataFrame: org.apache.spark.sql.DataFrame = [name: string, songs: map<string,string>]


In [45]:
simpleMapDataFrame.printSchema()
simpleMapDataFrame.show()

root
 |-- name: string (nullable = true)
 |-- songs: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+------------+--------------------+
|        name|               songs|
+------------+--------------------+
|     sublime|[good_song -> san...|
|prince_royce|[good_song -> dar...|
+------------+--------------------+



In [48]:
simpleMapDataFrame.select($"name",col("songs")("good_song").as("fun")).show()

+------------+-------------+
|        name|          fun|
+------------+-------------+
|     sublime|     santeria|
|prince_royce|darte un beso|
+------------+-------------+

